In [1]:
!pip install transformers


In [2]:
from transformers import BertTokenizer, BertForQuestionAnswering
import torch

# Load the BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('deepset/bert-base-cased-squad2')
model = BertForQuestionAnswering.from_pretrained('deepset/bert-base-cased-squad2')
import torch.optim as optim

# Assuming you already defined your model and criterion

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Example: Adam optimizer with learning rate 0.001

# Rest of your training loop



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Define the input text and question
text = "In a shocking turn of events, it was revealed that the once respected businessman had been embezzling funds from the company for years. Despite his deceit, many of his employees expressed their loyalty and support for him."
question = "Who was revealed to have been embezzling funds from the company?"


# Encode the input text and question, and get the scores for each word in the text
input = tokenizer(question, text,  return_tensors="pt")

output = model(**input)

# Find the words in the text that corresponds to the highest start and end scores
# with a torch.no_grad():
#     outputs = model(**inputs)
start_index = output.start_logits.argmax()
end_index = output.end_logits.argmax() + 1

# Extract the span of words as the answer
answer = tokenizer.decode(input.input_ids[0, start_index:end_index])
print(answer)


the once respected businessman


In [ ]:
data = [
                 ("I want to improve my strength. What exercises do you recommend?", "Strength training exercises like squats, deadlifts, and bench presses can help you build muscle. Do you have any equipment available or prefer bodyweight exercises?"),
                 ("I feel very tired during my period. Should I still exercise?", "It's important to listen to your body during your period. Light exercises like walking or gentle yoga can help alleviate discomfort. However, if you're feeling particularly fatigued, it's okay to take a break."),
                 ("How can I improve my flexibility?", "Incorporating stretching exercises like yoga or Pilates into your routine can help improve flexibility. Would you like some specific stretches to try?"),
                ("I'm feeling overwhelmed by all the conflicting information about exercise. How can I determine what's best for me?", "It's common to feel overwhelmed by the abundance of information available about exercise and fitness. Start by focusing on the basics: regular physical activity, a balanced diet, and listening to your body's signals. Experiment with different types of exercises and routines to see what works best for you, and don't hesitate to seek guidance from a qualified fitness professional if you need help."),
]

questions = [item[0] for item in data]
answers = [item[1] for item in data]

from transformers import BertTokenizer, BertForSequenceClassification
import torch
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# Convert the data to input format for BERT
question_input_ids = []
question_attention_masks = []
for i in range(len(questions)):
    # Tokenize the question
    question_tokens = tokenizer.tokenize(questions[i])

    max_length = 512
    # Pad the input tokens
    question_tokens = question_tokens + [tokenizer.pad_token] * (max_length - len(question_tokens))

    # Create the input ids for the BERT model
    question_input_ids.append(tokenizer.convert_tokens_to_ids(question_tokens))

    # Create the attention masks for the input tokens
    question_attention_masks.append([1 if token != tokenizer.pad_token else 0 for token in question_tokens])

answer_input_ids = []
answer_attention_masks = []
for i in range(len(answers)):
    # Tokenize the answer
    answer_tokens = tokenizer.tokenize(answers[i])

    # Pad the input tokens
    answer_tokens = answer_tokens + [tokenizer.pad_token] * (max_length - len(answer_tokens))

    # Create the input ids for the BERT model
    answer_input_ids.append(tokenizer.convert_tokens_to_ids(answer_tokens))

    # Create the attention masks for the input tokens
    answer_attention_masks.append([1 if token != tokenizer.pad_token else 0 for token in answer_tokens])

# Concatenate the question and answer input lists
input_ids = question_input_ids + answer_input_ids
attention_masks = question_attention_masks + answer_attention_masks

# Convert the input ids and attention masks to tensors
input_ids = torch.tensor(input_ids).to(device)
attention_masks = torch.tensor(attention_masks).to(device)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Define the criterion
criterion = torch. nn.CrossEntropyLoss()

def calculate_accuracy(predictions, labels):
    correct_predictions = 0
    for i in range(len(predictions)):
        if predictions[i] == labels[i]:
            correct_predictions += 1
    return correct_predictions / len(predictions)

# Set the number of epochs
num_epochs = 20

# Set the labels for the data
labels = [0 if i < len(questions) else 1 for i in range(len(questions) + len(answers))]
labels = torch.tensor(labels).to(device)

# Set the training loop
for epoch in range(num_epochs):

    # Set the training mode
    model.train()

    # Clear the gradients
    model.zero_grad()

    # Forward pass
    output = model(input_ids, attention_mask=attention_masks)

    # Calculate the loss
    loss = criterion(output[0], labels)

    # Backward pass
    loss.backward()

    # Update the parameters
    optimizer.step()

    # Print the loss and accuracy
    print("Epoch {}/{} - Loss: {:.5f} - Accuracy: {:.5f}".format(epoch + 1, num_epochs, loss.item(), calculate_accuracy(output[0].argmax(dim=1).cpu().numpy(), labels.cpu().numpy())))


In [ ]:
# Set the model to eval mode
model.eval()

# Define the input
input_text = "What are some exercises I can do to improve my posture?"

# Tokenize the input
input_tokens = tokenizer.tokenize(input_text)

# Pad the input tokens
input_tokens = input_tokens + [tokenizer.pad_token] * (max_length - len(input_tokens))

# Convert the input tokens to input ids
input_ids = tokenizer.convert_tokens_to_ids(input_tokens)

# Create the attention mask for the input
attention_mask = [1 if token != tokenizer.pad_token else 0 for token in input_tokens]

# Convert the input ids and attention mask to tensors
input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)
attention_mask = torch.tensor(attention_mask).unsqueeze(0).to(device)

# Get the model output
output = model(input_ids, attention_mask=attention_mask)

# Get the predicted label
prediction = output[0].argmax(dim=1).item()

# Print the output
if prediction == 0:
    print("Question: {}".format(input_text))
else:
    print("Answer: {}".format(answers[prediction - 1]))



Answer: Exercises like rows, shoulder blade squeezes, and core strengthening exercises can help improve posture. Additionally, practicing good posture throughout the day is important.


In [3]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [8]:
model_save_path = '/gdrive/My Drive/model.pt'

# Save the model
torch.save(model.state_dict(), model_save_path)

In [4]:
model_load_path = '/gdrive/My Drive/model.pt'

# Load the model
model = BertForQuestionAnswering.from_pretrained('deepset/bert-base-cased-squad2')  # Instantiate your model class
model.load_state_dict(torch.load(model_load_path))

# Ensure the model is in evaluation mode
model.eval()

Some weights of the model checkpoint at deepset/bert-base-cased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

In [5]:
data = [
        ("I'm feeling stressed. Will exercise help?", "Exercise is a great way to reduce stress and improve mood. Physical activity releases endorphins, which are chemicals in the brain that act as natural painkillers and mood elevators. Try incorporating activities like walking, jogging, or yoga into your routine to help manage stress."),
        ("What are some exercises I can do to improve my posture?", "Exercises like rows, shoulder blade squeezes, and core strengthening exercises can help improve posture. Additionally, practicing good posture throughout the day is important."),
        ("I have trouble staying motivated to exercise regularly. Any tips?", "Setting specific, achievable goals, finding a workout buddy for accountability, and varying your routine with different types of exercises can help keep you motivated. Remember to focus on how exercise makes you feel, rather than just the end results."),
]

data = [
                   ("I'm not seeing the results I want from my workouts. What am I doing wrong?", "Several factors could be contributing to a lack of results, including not exercising with enough intensity or frequency, poor nutrition, inadequate recovery time, or unrealistic expectations. Consider evaluating your workout routine, nutrition habits, and lifestyle factors to identify areas for improvement."),
                 ("What should I eat before and after a workout?", "Before a workout, it's important to fuel your body with a combination of carbohydrates and protein for energy and muscle repair. Consider options like a banana with peanut butter, Greek yogurt with fruit, or a turkey sandwich on whole-grain bread. After a workout, aim to consume a balanced meal or snack containing protein and carbohydrates to replenish energy stores and support muscle recovery."),
                 ("I'm traveling and don't have access to a gym. What are some exercises I can do without equipment?", "Bodyweight exercises like push-ups, squats, lunges, and planks are great options for working out while traveling. You can also incorporate resistance bands or use household items like water bottles or backpacks as makeshift weights."),

]

data = [
                    ("I'm pregnant. What exercises are safe for me to do?", "Congratulations! Low-impact exercises like swimming, walking, and prenatal yoga are generally safe during pregnancy. It's best to consult with your healthcare provider before starting any new exercise routine."),
                 ("I have a knee injury. What exercises can I do to strengthen my knee?", "Exercises that focus on strengthening the muscles around the knee, such as leg lifts and hamstring curls, can help improve knee stability. It's important to avoid exercises that cause pain or discomfort. Have you consulted with a physical therapist for personalized recommendations?"),
                 ("I'm trying to lose weight. What types of exercises are best for burning calories?", "Cardiovascular exercises like running, cycling, and HIIT workouts are effective for burning calories. Additionally, incorporating strength training exercises can help increase muscle mass and boost metabolism."),

]

data = [

    ("I'm in my 60s and want to stay active. What exercises are suitable for seniors?",
     "Seniors can benefit from activities like walking, water aerobics, gentle yoga, tai chi, and resistance training using light weights or resistance bands to maintain mobility, strength, and balance."),

    ("What exercises can teenagers do to stay active?",
     "Teenagers can participate in team sports, individual sports, strength training, yoga, and other activities to stay active and healthy during their adolescent years."),

    ("I'm in my 20s and want to improve my fitness. What exercises do you recommend?",
     "In your 20s, you can focus on a variety of exercises including strength training, cardio (such as running, cycling, or swimming), HIIT workouts, and flexibility training to improve overall fitness.")


]

data = [
    ("I'm trying to lose weight. What types of exercises are best for burning calories?", "Cardiovascular exercises like running, cycling, and HIIT workouts are effective for burning calories. Additionally, incorporating strength training exercises can help increase muscle mass and boost metabolism."),
    ("I suffer from back pain. What exercises can I do to alleviate it?", "Exercises such as gentle yoga, Pilates, and core strengthening exercises can help alleviate back pain by improving posture and strengthening the muscles that support the spine."),
    ("I have limited time for exercise. What are some effective quick workouts?", "High-intensity interval training (HIIT) workouts are great for maximizing calorie burn in a short amount of time. Tabata workouts, circuit training, and bodyweight exercises like burpees and jumping jacks can also provide a quick and effective workout.")
]

questions = [item[0] for item in data]
answers = [item[1] for item in data]

from transformers import BertTokenizer, BertForSequenceClassification
import torch
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# Convert the data to input format for BERT
question_input_ids = []
question_attention_masks = []
for i in range(len(questions)):
    # Tokenize the question
    question_tokens = tokenizer.tokenize(questions[i])

    max_length = 512
    # Pad the input tokens
    question_tokens = question_tokens + [tokenizer.pad_token] * (max_length - len(question_tokens))

    # Create the input ids for the BERT model
    question_input_ids.append(tokenizer.convert_tokens_to_ids(question_tokens))

    # Create the attention masks for the input tokens
    question_attention_masks.append([1 if token != tokenizer.pad_token else 0 for token in question_tokens])

answer_input_ids = []
answer_attention_masks = []
for i in range(len(answers)):
    # Tokenize the answer
    answer_tokens = tokenizer.tokenize(answers[i])

    # Pad the input tokens
    answer_tokens = answer_tokens + [tokenizer.pad_token] * (max_length - len(answer_tokens))

    # Create the input ids for the BERT model
    answer_input_ids.append(tokenizer.convert_tokens_to_ids(answer_tokens))

    # Create the attention masks for the input tokens
    answer_attention_masks.append([1 if token != tokenizer.pad_token else 0 for token in answer_tokens])

# Concatenate the question and answer input lists
input_ids = question_input_ids + answer_input_ids
attention_masks = question_attention_masks + answer_attention_masks

# Convert the input ids and attention masks to tensors
input_ids = torch.tensor(input_ids).to(device)
attention_masks = torch.tensor(attention_masks).to(device)



tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [6]:
# Define the criterion
criterion = torch. nn.CrossEntropyLoss()

def calculate_accuracy(predictions, labels):
    correct_predictions = 0
    for i in range(len(predictions)):
        if predictions[i] == labels[i]:
            correct_predictions += 1
    return correct_predictions / len(predictions)

# Set the number of epochs
num_epochs = 20

# Set the labels for the data
labels = [0 if i < len(questions) else 1 for i in range(len(questions) + len(answers))]
labels = torch.tensor(labels).to(device)

# Set the training loop
for epoch in range(num_epochs):

    # Set the training mode
    model.train()

    # Clear the gradients
    model.zero_grad()

    # Forward pass
    output = model(input_ids, attention_mask=attention_masks)

    # Calculate the loss
    loss = criterion(output[0], labels)

    # Backward pass
    loss.backward()

    # Update the parameters
    optimizer.step()

    # Print the loss and accuracy
    print("Epoch {}/{} - Loss: {:.5f} - Accuracy: {:.5f}".format(epoch + 1, num_epochs, loss.item(), calculate_accuracy(output[0].argmax(dim=1).cpu().numpy(), labels.cpu().numpy())))


Epoch 1/20 - Loss: 6.27761 - Accuracy: 0.00000
Epoch 2/20 - Loss: 6.22513 - Accuracy: 0.00000
Epoch 3/20 - Loss: 6.38941 - Accuracy: 0.00000
Epoch 4/20 - Loss: 6.21098 - Accuracy: 0.00000
Epoch 5/20 - Loss: 6.34282 - Accuracy: 0.00000
Epoch 6/20 - Loss: 6.22678 - Accuracy: 0.00000
Epoch 7/20 - Loss: 6.11601 - Accuracy: 0.00000
Epoch 8/20 - Loss: 6.33411 - Accuracy: 0.00000
Epoch 9/20 - Loss: 6.18288 - Accuracy: 0.00000
Epoch 10/20 - Loss: 6.14049 - Accuracy: 0.00000
Epoch 11/20 - Loss: 6.26720 - Accuracy: 0.00000
Epoch 12/20 - Loss: 6.20726 - Accuracy: 0.00000
Epoch 13/20 - Loss: 6.32181 - Accuracy: 0.00000
Epoch 14/20 - Loss: 6.40035 - Accuracy: 0.00000
Epoch 15/20 - Loss: 6.54740 - Accuracy: 0.00000
Epoch 16/20 - Loss: 6.32883 - Accuracy: 0.00000
Epoch 17/20 - Loss: 6.24288 - Accuracy: 0.00000
Epoch 18/20 - Loss: 6.32151 - Accuracy: 0.00000
Epoch 19/20 - Loss: 6.25790 - Accuracy: 0.00000
Epoch 20/20 - Loss: 6.27338 - Accuracy: 0.00000
